In [1]:
import sys
print('python', sys.version)

import numpy as np
print('numpy', np.__version__)

import pandas as pd
print('pandas', pd.__version__)

import matplotlib as mpl
print('matplotlib', mpl.__version__)

import matplotlib.pyplot as plt
import scipy.stats as sci
import glob

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

python 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
numpy 1.19.1
pandas 1.1.1
matplotlib 3.3.1


## comparing to misl

### processing GIs in Misl

In [2]:
def eval_hypergeo(HT,AL,GS,SP):
    hypergeo_pval=sci.hypergeom.sf(len(HT),len(AL),len(GS),len(SP))
    return [len(HT),len(AL),len(GS),len(SP), hypergeo_pval]

In [3]:
# mapping human gene symbol to entrez
dat=pd.read_table('data_ori/Sym2Entrez.txt',sep='\t')
dat.columns=['sym','entrez']
dat=dat.loc[dat['entrez'].notnull()]
dat=dat.loc[dat['sym'].notnull()]
dat['entrez']=dat['entrez'].astype(int)

e2s=dat.set_index('entrez')
e2s_dic=e2s['sym'].to_dict() # e2s_dic={1: 'A1BG', 503538: 'A1BG-AS1',...}
    
def merge_two_gene1(row):
    g1=row['mut_gene']
    g2=row['dep_gene']

    return g1+'_'+g2


def get_MSL_all_pair(MSL_ori):
    MSL_flat=pd.DataFrame(columns=['mut','SL'])
    jj=0
    for ii in MSL_ori.index:
        mut,num,SLs=MSL_ori.loc[ii]
        if mut not in e2s_dic.values():
            continue
        for SL in SLs.split(', '):
            if SL not in e2s_dic.values():
                continue
            MSL_flat.loc[jj]=[mut,SL]
            jj+=1
    
    return MSL_flat

def get_num_of_occuracne(MSL_entz):
    MSL_cnt=MSL_entz.groupby(MSL_entz.columns.tolist(),as_index=False).size()
    MSL_cnt['mg']=MSL_cnt['mut']+'_'+MSL_cnt['SL']
    
    return MSL_cnt

def get_GS():
    MSL_ori=pd.read_table('data_ori/gold_standard_Set/SL_human (MISL).txt',header=None,sep='\t')
    MSL_ori.columns=['mut','num_of_SL','SL']
    MSL_flat=get_MSL_all_pair(MSL_ori)
    return set(MSL_flat['mut']+'_'+MSL_flat['SL'])

def get_ALL(tp,W):
    a2=pd.read_table('result_SL/{}_M003_I10_{}_Exp_fdr.txt'.format(tp,W), sep='\t', usecols=[0,1,2,4])
    a2['mg']=a2.apply(merge_two_gene1, axis=1)
    a2=a2.drop(columns=['mut_gene','dep_gene'])
    return a2

def f1(r1):
    d1=r1['DST=1'][1:-1].replace("'","").split(', ')
    d1=[ii for ii in d1 if ii!='']
    d2=r1['DST=2'][1:-1].replace("'","").split(', ')
    d2=[ii for ii in d2 if ii!='']
    it=r1['INIT.'][1:-1].replace("'","").split(', ')
    it=[ii for ii in it if ii!='']
    
    d2m=d1+d2
    itm=d1+d2+it
    
    return pd.Series([itm, d2m, d1],index=['it','d2','d1'])

def get_SP(file_name):
    crp_KEGG=pd.read_table(file_name,sep='\t',index_col=0)
    crp_KEGG_merged=crp_KEGG.apply(f1, axis=1).reset_index().rename(columns={'index':'MUT'})
    crp_KEGG_melt=crp_KEGG_merged.melt(id_vars='MUT', var_name='RP', value_name='SP')

    crp_KEGG_m2=pd.DataFrame(columns=['RP','MUT','SP','MUT_SP'])

    ind=0
    for idx in crp_KEGG_melt.index:
        row=crp_KEGG_melt.loc[idx]
        for sp in row['SP']:
            crp_KEGG_m2.loc[ind]=[row['RP'],row['MUT'],sp, row['MUT']+'_'+sp]
            ind+=1
            
    crp_KEGG_m2=crp_KEGG_m2.drop(columns=['MUT','SP'])        
    return crp_KEGG_m2

In [4]:
def check_tval_pos(mg, AL_df_subset):
    if AL_df_subset.loc[AL_df_subset['mg']==mg, 't_val'].iloc[0]>0:
        return True
    else:
        return False
    
def func_check_tval_pos(SP_df):
    AL_df_subset=AL_df.loc[AL_df['mg'].isin(set(SP_df['MUT_SP']))]
    SP_pos_df=pd.DataFrame(columns=['RP','MUT_SP'])
    for ind, row in SP_df.iterrows():
        if check_tval_pos(row['MUT_SP'], AL_df_subset):
            SP_pos_df.loc[ind]=list(row)
    SP_pos_df.reset_index(drop=True, inplace=True)
    return SP_pos_df

In [5]:
tp2tp={'CRISPER':'crp','shRNA':'shr'}

GS_init=get_GS()

res_df=pd.DataFrame(columns=['screening','zero_exp','NET','RP','HIT','ALL','GS','SP','hyp_pval'])
## WO ==> including zero expression
## WI ==> excluding zero expression
ind=0
HT_dic={}

for TP in ['CRISPER','shRNA']:
    for W in ['WO','WI']:
        AL_df=get_ALL(TP, W)
        AL=set(AL_df['mg'])
        GS=GS_init & AL
        SP=set(AL_df.loc[(AL_df['fdr']<0.2)&(AL_df['t_val']>0), 'mg'])
        HT=GS&SP
        
        HT_dic[(TP,W, 0,'HT')]=HT
        HT_dic[(TP,W, 0,'SP')]=SP
        
        res=eval_hypergeo(HT,AL,GS,SP)
        res_df.loc[ind]=[TP,{'WO':'IZ','WI':'EZ'}[W],np.nan,np.nan]+res
        ind+=1
        
    for NET in ['KEGG','PPI']:
        print('########', TP, NET)
        SP_df=get_SP('result_main/{}_diff_table_{}.txt'.format(tp2tp[TP], NET))
        print(SP_df)
        SP_pos_df=func_check_tval_pos(SP_df)
        print(SP_pos_df)

        for RP in ['it','d2','d1']:
            SP=set(SP_pos_df.loc[SP_pos_df['RP']==RP,'MUT_SP'])
            HT=SP&GS

            HT_dic[(TP,NET, RP,'HT')]=HT
            HT_dic[(TP,NET, RP,'SP')]=SP
            
            res=eval_hypergeo(HT,AL,GS,SP)
            res_df.loc[ind]=[TP,{'WO':'IZ','WI':'EZ'}[W],NET,RP]+res
            ind+=1

######## CRISPER KEGG
     RP        MUT_SP
0    it     TP53_ARF6
1    it     TP53_MDM2
2    it   TP53_CDKN1A
3    it     TP53_AFDN
4    it     TP53_MDM4
..   ..           ...
295  d1  PRRC2A_IGF1R
296  d1  SREBF1_IGF1R
297  d1   SREBF1_IRS1
298  d1  ZNF672_FGFR1
299  d1   ZNF672_IRS1

[300 rows x 2 columns]
     RP         MUT_SP
0    it      TP53_ARF6
1    it    TP53_CDKN1A
2    it      TP53_TP53
3    it       TP53_ATM
4    it     TP53_SIPA1
..   ..            ...
101  d1      NRAS_RAF1
102  d1       RB1_SKP2
103  d1      RB1_CKS1B
104  d1  PIK3CA_PIK3CA
105  d1      MYO6_TP53

[106 rows x 2 columns]
######## CRISPER PPI
      RP         MUT_SP
0     it      TP53_RPL7
1     it    TP53_MAD2L2
2     it     TP53_DHX30
3     it     TP53_RPL19
4     it    TP53_QRICH1
...   ..            ...
1574  d1     MCTP2_EGFR
1575  d1      MYO6_MDM2
1576  d1      MYO6_TP53
1577  d1    RPTOR_IGF1R
1578  d1  RPTOR_HSP90B1

[1579 rows x 2 columns]
     RP        MUT_SP
0    it   TP53_MAD2L2
1    it    T

In [6]:
res_df.to_csv('eval_result/MISL_results.txt', sep='\t')